In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob

%matplotlib inline

In [ ]:
N_COLUMNS = 36 # should infer from df number of columns

ROBOT_INFO_SIZE = 3
DETECTION_INFO_SIZE = 1
CLUSTER_INFO_SIZE = 4

N_CLUSTERS = int((N_COLUMNS - ROBOT_INFO_SIZE - DETECTION_INFO_SIZE) / CLUSTER_INFO_SIZE)

# Load Data

In [ ]:
DETECTION_FOLDER = 'experiments/detection_exps/'
NO_DETECTION_FOLDER = 'experiments/no_detection_exps/'

In [ ]:
def load_robot_data(file_path):
    df = pd.read_csv(file_path, header=None)
    column_names = ['robot_x', 'robot_y', 'robot_angle']
    for i in range(N_CLUSTERS):
        column_names.extend([f'cluster{i}_x', f'cluster{i}_y', f'cluster{i}_angle', f'cluster{i}_weight'])
    column_names.append('has_detection')
    df.columns = column_names
    return df

detection_dfs = dict()
for file_name in glob.glob(DETECTION_FOLDER + '*.csv'):
    df = load_robot_data(file_name)
    name = file_name[27:33] + file_name[-12:-4]
    detection_dfs[name] = df

no_detection_dfs = dict()
for file_name in glob.glob(NO_DETECTION_FOLDER + '*.csv'):
    df = load_robot_data(file_name)
    name = file_name[30:36] + file_name[-12:-4]
    no_detection_dfs[name] = df

# Clusters error

In [ ]:
def compute_clusters_errors(df):
    for cluster_id in range(N_CLUSTERS):
        diff_x_square = (df[f"robot_x"] - df[f"cluster{cluster_id}_x"])**2
        diff_y_square = (df[f"robot_y"] - df[f"cluster{cluster_id}_y"])**2
        df[f"cluster{cluster_id}_error"] = np.sqrt(diff_x_square + diff_y_square)
    return df

for k, v in detection_dfs.items():
    detection_dfs[k] = compute_clusters_errors(v)

for k, v in no_detection_dfs.items():
    no_detection_dfs[k] = compute_clusters_errors(v)

# Plot results

In [ ]:
EXPERIMENT_TIME_S = 60
CELLS_TO_METERS = 10

def _cells_to_meters(cells):
    return cells / CELLS_TO_METERS

## Top N (accumulated) clusters error

In [ ]:
def plot_top_n_accumulated_cluster_errors(df, title):
    t = np.linspace(1, EXPERIMENT_TIME_S, len(df))

    N_CLUSTERS_TO_PLOT = 3
    plt.figure(figsize=(15, 2), facecolor='w')
    plt.title('Error between estimated cluster position and ground truth [m].')

    weights = []
    for cluster_id in range(N_CLUSTERS):
        df[f"cluster{cluster_id}_w_mean"] = df[f"cluster{cluster_id}_weight"].mean()
        weights.append(df[f"cluster{cluster_id}_w_mean"].iloc[0])

    weights_to_plot = sorted(weights, reverse=True)[:N_CLUSTERS_TO_PLOT]
    for cluster_id in range(N_CLUSTERS):
        weight = df[f"cluster{cluster_id}_w_mean"].iloc[0]
        if weight in weights_to_plot: 
            plt.plot(t, _cells_to_meters(
                df[f"cluster{cluster_id}_error"]), alpha=0.8, label=f"cluster {cluster_id}")

    plt.plot(t, df.has_detection * 10, alpha=0.5, label="has detection")
    plt.title(title)
    plt.xlabel('Time [s]')
    plt.legend()
    plt.grid()
    plt.show()

### Detection 

In [ ]:
for k, v in detection_dfs.items():
    plot_top_n_accumulated_cluster_errors(v, k)

### No Detection

In [ ]:
for k, v in no_detection_dfs.items():
    plot_top_n_accumulated_cluster_errors(v, k)

## Best (instantaneous) cluster error

In [ ]:
def plot_best_instantaneous_cluster_error(df, title):
    def _compute_best_cluster_and_its_error(row):
        best_cluster_id = np.argmax(row.filter(like='_weight'))
        row['best_cluster_id'] = best_cluster_id
        row['best_cluster_error'] = row[f'cluster{best_cluster_id}_error']
        return row 

    df = df.apply(_compute_best_cluster_and_its_error, axis=1)

    t = np.linspace(1, EXPERIMENT_TIME_S, len(df))

    plt.figure(figsize=(15, 2))
    plt.title(title)
    plt.plot(t, _cells_to_meters(df['best_cluster_error']), alpha=0.8, label='robot')
    plt.plot(t, df.has_detection * 10, alpha=0.5, label="has detection")
    plt.xlabel('Time [s]')
    plt.legend()
    plt.grid()
    plt.show()
    
    return df

### Detection

In [ ]:
for k, v in detection_dfs.items():
    detection_dfs[k] = plot_best_instantaneous_cluster_error(v, k)

### No Detection

In [ ]:
for k, v in no_detection_dfs.items():
    no_detection_dfs[k] = plot_best_instantaneous_cluster_error(v, k)

## Best (instantaneous) cluster error accumulated

In [ ]:
def plot_best_instantaneous_cluster_error_accumulated(accumulated_error, title, experiment_size):
    t = np.linspace(1, EXPERIMENT_TIME_S, experiment_size)

    plt.figure(figsize=(15, 2))
    plt.title(title)
    plt.plot(t, _cells_to_meters(accumulated_error), alpha=0.8, label='robot')
    plt.xlabel('Time [s]')
    plt.legend()
    plt.grid()
    plt.show()

### Detection

In [ ]:
EXPERIMENT_LEN_DETECT = 300

robot1_accumulated_detection = np.zeros(EXPERIMENT_LEN_DETECT)
robot2_accumulated_detection = np.zeros(EXPERIMENT_LEN_DETECT)
for k, v in detection_dfs.items():
    if 'robot1' in k:
        robot1_accumulated_detection += v.best_cluster_error[:EXPERIMENT_LEN_DETECT]
    elif 'robot2' in k:
        robot2_accumulated_detection += v.best_cluster_error[:EXPERIMENT_LEN_DETECT]
        
plot_best_instantaneous_cluster_error_accumulated(
    robot1_accumulated_detection, 'Robot1 detection', EXPERIMENT_LEN_DETECT)
plot_best_instantaneous_cluster_error_accumulated(
    robot2_accumulated_detection, 'Robot2 detection', EXPERIMENT_LEN_DETECT)

### No detection

In [ ]:
EXPERIMENT_LEN_NO_DETECT = 290

robot1_accumulated_no_detection = np.zeros(EXPERIMENT_LEN_NO_DETECT)
robot2_accumulated_no_detection = np.zeros(EXPERIMENT_LEN_NO_DETECT)
for k, v in no_detection_dfs.items():
    if 'robot1' in k:
        robot1_accumulated_no_detection += v.best_cluster_error[:EXPERIMENT_LEN_NO_DETECT]
    elif 'robot2' in k:
        robot2_accumulated_no_detection += v.best_cluster_error[:EXPERIMENT_LEN_NO_DETECT]
        
plot_best_instantaneous_cluster_error_accumulated(
    robot1_accumulated_no_detection, 'Robot1 no detection', EXPERIMENT_LEN_NO_DETECT)
plot_best_instantaneous_cluster_error_accumulated(
    robot2_accumulated_no_detection, 'Robot2 no detection', EXPERIMENT_LEN_NO_DETECT)

### Comparison Detection vs No Detection

In [ ]:
def plot_best_instantaneous_cluster_error_accumulated(
    accumulated_error_detect, accumulated_error_no_detect, title, experiment_size):
#     t = np.linspace(1, EXPERIMENT_TIME_S, experiment_size)

    plt.figure(figsize=(15, 2))
    plt.title(title)
    plt.plot(_cells_to_meters(accumulated_error_detect), alpha=0.8, label='robot_detect')
    plt.plot(_cells_to_meters(accumulated_error_no_detect), alpha=0.8, label='robot_no_detect')
    plt.xlabel('Ticks')
    plt.legend()
    plt.grid()
    plt.show()
    
plot_best_instantaneous_cluster_error_accumulated(
    robot1_accumulated_detection, robot1_accumulated_no_detection, 'Robot1', EXPERIMENT_LEN_DETECT)
plot_best_instantaneous_cluster_error_accumulated(
    robot2_accumulated_detection, robot2_accumulated_no_detection, 'Robot2', EXPERIMENT_LEN_NO_DETECT)

Done!